# Make color-prior object dataset
This notebook creates a dataset of real-world objects with **known color priors**.
Images are obtained through targeted queries in *query_google_images.ipynb* and segmented to produce binary foreground masks using *segment_outline.ipynb*.
Using these masks, foreground and background regions are colored independently, enabling controlled manipulations of object color while preserving object identity.
Recoloring is implemented via *recolor_images.py*.
Color priors are model-specific and can be queried using *model_priors.py*.

In [ ]:
%reload_ext autoreload
%autoreload 2

from transformers import (
    BitsAndBytesConfig,
    LlavaNextProcessor,
    LlavaNextForConditionalGeneration
)
import torch
import pandas as pd
import re
import gc
import numpy as np
from tqdm import tqdm
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt

ROOT = Path().resolve().parents[0]
sys.path.insert(0, str(ROOT))

from model_evaluation.test_MLLMs import run_vlm_evaluation
from making_color_images.model_priors import TorchColorPriors, GPTColorPriors
from making_color_images.plot_variants import collect_variants_for, show_variants_grid, plot_vlm_performance, variant_label
from making_color_images.recolor_images import generate_variants, resize_all_images_and_masks


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

fontsize = 14

In [9]:
# Set a specific seed for reproducibility
SEED = 42
rng = np.random.default_rng(SEED)

# Setting the seed for PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # If using GPU

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True
)

torch.cuda.empty_cache()
gc.collect()

Using device: cuda


20

In [5]:
# Base data folder
WORK = Path(os.environ.get("WORK", Path.cwd()))
DATA = WORK / "color-concept-entanglement" / "data"

FRUIT = DATA / "fruit"
OUTLINES = DATA / "color_images"

RESIZED_IMGS = DATA / "resized_images"
RESIZED_MASKS = DATA / "resized_cv_masks"
RESIZED_IMGS.mkdir(parents=True, exist_ok=True)
RESIZED_MASKS.mkdir(parents=True, exist_ok=True)

In [6]:
# Define the percentage grid for coloring
pct_schedule = list(range(0, 11, 1)) + list(range(20, 60, 10)) + list([55]) + list(range(60, 110, 10))
print("Percentage grid:", pct_schedule)

Percentage grid: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 30, 40, 50, 55, 60, 70, 80, 90, 100]


## 1. LLaVA-NeXT

### 1.1. Load model and base dataframe

In [10]:
model_name = "llava-v1.6-mistral-7b-hf"
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained(
    "llava-hf/llava-v1.6-mistral-7b-hf", dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", quantization_config=bnb_config
).to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Load dataframe with outline images and masks
df = pd.read_csv(DATA / "segmented_images.csv")
display(df)

,correct_answer,incorrect_answer,object,image_url,image_path,gpt_response,score,watermark,num_objects,cv_mask_path,sam_mask_path
0,['yellow'],purple,sunflower,https://thumbs.dreamstime.com/b/black-outline-...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
1,"['brown', 'black', 'orange']",green,tabby,https://img.recraft.ai/5P-WlUIGI2MwJC2v_JyrGkp...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
2,"['pink', 'white']",green,spoonbill,https://img.freepik.com/premium-vector/drawing...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
3,['white'],green,Samoyed,https://img.freepik.com/premium-photo/black-wh...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
4,"['grey', 'black']",red,Norwegian elkhound,https://t3.ftcdn.net/jpg/16/85/30/00/360_F_168...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
...,...,...,...,...,...,...,...,...,...,...,...
215,"['gold', 'silver']",green,crown,https://i.pinimg.com/736x/32/52/35/325235d85fb...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",42.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
216,"['orange', 'white']",purple,pill bottle,https://media.istockphoto.com/id/470313936/vec...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",35.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
217,['black'],pink,dial telephone,https://png.pngtree.com/png-clipart/20230811/o...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",35.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
218,['black'],pink,digital watch,https://img.freepik.com/premium-vector/digital...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",34.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...


### 1.2. Get model-specific color priors

In [12]:
# Initalize ModelColorPriors
priors = TorchColorPriors(
    processor=processor,
    model=model,
    data_folder=DATA,
    device=device
)

In [ ]:
# Generate new priors
# priors_df = priors.get_model_color_priors(df, save=True)

In [13]:
# Checkpoint: display the generated priors
priors_df = priors.load_model_priors()
display(priors_df[['object', 'correct_answer', 'dummy_priors', 'image_priors']])

,object,correct_answer,dummy_priors,image_priors
0,sunflower,[yellow],"[yellow, brown, green]","[yellow, black, white]"
1,tabby,"[brown, black, orange]","[black, grey, white]","[black, grey, white]"
2,spoonbill,"[pink, white]","[pink, white, black]","[white, black, grey]"
3,Samoyed,[white],"[white, cream, light brown]","[white, cream, light grey]"
4,Norwegian elkhound,"[grey, black]","[grey, black, brown]","[white, grey, black]"
...,...,...,...,...
215,crown,"[gold, silver]","[gold, silver, white]","[gold, silver, black]"
216,pill bottle,"[orange, white]","[blue, white, green]","[white, blue, green]"
217,dial telephone,[black],"[black, white, grey]","[black, white, grey]"
218,digital watch,[black],"[black, grey, blue]","[black, silver, white]"


In [14]:
# Pick a primary color prior, overwrite correct_answer, and drop all rows where the prior is None
DUMMY = False  # Set to True to use dummy priors, False for image priors
suffix = "dummy_priors" if DUMMY else "image_priors"
priors_df["prior"], priors_df["prior_changed"] = priors.pick_primary_color(priors_df, column=suffix)

[INFO] tabby: replaced 'black' with 'grey'
[INFO] spoonbill: replaced 'white' with 'grey'
[NULL] Samoyed: all priors invalid ['white', 'cream', 'light grey'] to NaN
[INFO] Norwegian elkhound: replaced 'white' with 'grey'
[INFO] hare: replaced 'white' with 'grey'
[NULL] car wheel: all priors invalid ['black', 'silver', 'white'] to NaN
[INFO] dandelion: replaced 'white' with 'yellow'
[INFO] bat_(animal): replaced 'black' with 'grey'
[INFO] buffalo: replaced 'black' with 'grey'
[INFO] ground beetle: replaced 'black' with 'brown'
[INFO] miniature pinscher: replaced 'white' with 'brown'
[INFO] oriole: replaced 'black' with 'grey'
[INFO] wombat: replaced 'black' with 'grey'
[INFO] tiger: replaced 'black' with 'orange'
[INFO] Chesapeake Bay retriever: replaced 'black' with 'brown'
[INFO] gorilla: replaced 'black' with 'brown'
[INFO] golf ball: replaced 'white' with 'yellow'
[INFO] spotted salamander: replaced 'black' with 'brown'
[INFO] american football: replaced 'white' with 'brown'
[NULL] 

In [15]:
print(f"Number of objects with valid model color priors: {len(priors_df['prior'].dropna())}")
print(f"Number of objects where the original prior was changed to be valid: {priors_df['prior_changed'].sum()}")
df["correct_answer"] = priors_df["prior"]

df.dropna(subset=["correct_answer"], axis=0, inplace=True)

#priors.analyze_differences(priors_df)
display(df)
df["correct_answer"].value_counts()

Number of objects with valid model color priors: 196
Number of objects where the original prior was changed to be valid: 146


,correct_answer,incorrect_answer,object,image_url,image_path,gpt_response,score,watermark,num_objects,cv_mask_path,sam_mask_path
0,yellow,purple,sunflower,https://thumbs.dreamstime.com/b/black-outline-...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
1,grey,green,tabby,https://img.recraft.ai/5P-WlUIGI2MwJC2v_JyrGkp...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
2,grey,green,spoonbill,https://img.freepik.com/premium-vector/drawing...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
4,grey,red,Norwegian elkhound,https://t3.ftcdn.net/jpg/16/85/30/00/360_F_168...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
5,red,yellow,lobster,https://media.istockphoto.com/id/848729864/vec...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
...,...,...,...,...,...,...,...,...,...,...,...
213,grey,yellow,cloud,https://i.pinimg.com/736x/82/89/8a/82898af848f...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",42.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
214,red,blue,apple,https://i.pinimg.com/736x/48/39/63/48396351091...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",42.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
216,blue,purple,pill bottle,https://media.istockphoto.com/id/470313936/vec...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",35.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
217,grey,pink,dial telephone,https://png.pngtree.com/png-clipart/20230811/o...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",35.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...


correct_answer
grey      59
brown     48
red       26
green     26
yellow    16
orange    12
blue       7
pink       1
purple     1
Name: count, dtype: int64

### 1.3. Color black-and-white base images with color prior

In [16]:
df = resize_all_images_and_masks(
    df,
    img_out_folder=RESIZED_IMGS,
    mask_out_folder=RESIZED_MASKS,
    target_size=512,              # or 256, 384, 1024
    mask_column="cv_mask_path",   
    img_column="image_path"
)


Resizing all: 100%|██████████| 196/196 [00:10<00:00, 19.27it/s]

Resized 196 images.


In [17]:
# Set coloring modes
coloring_modes = ["sequential"] #, "independent"]

OUT_COLOR = OUTLINES / model_name / suffix
OUT_COLOR.mkdir(parents=True, exist_ok=True)
print("Output folder for colored images:", OUT_COLOR)

Output folder for colored images: /mnt/lustre/work/eickhoff/esx061/color-concept-entanglement/data/color_images/llava-v1.6-mistral-7b-hf/image_priors


In [ ]:
# Generate variants for all images in df
for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating color variants"):
    for color_mode in coloring_modes:
        paths = generate_variants(
            row=row,
            target_color=row['correct_answer'],
            out_dir=OUT_COLOR,
            rng=rng,
            use_patches=True,
            patch_size=16,
            step_size=10,
            mode=color_mode,
            pct_schedule=pct_schedule
        )

In [ ]:
# Define which range of image variants to plot
tail_range = list(range(0, 6)) + list(range(95, 101))
ten_increase = list(range(0, 110, 10))
compact = [0, 10, 50, 90, 100]

In [ ]:
for idx, row in df.head(5).iterrows():
    for color_mode in coloring_modes:
        show_variants_grid(row['image_path'], row['correct_answer'], OUT_COLOR, color_mode=color_mode, pct_range=ten_increase)

## 2. GPT-4o

### 2.1. Load model and base dataframe

In [18]:
# Load segmented images dataframe again for GPT
df_GPT = pd.read_csv(DATA / "segmented_images.csv")
display(df_GPT)

,correct_answer,incorrect_answer,object,image_url,image_path,gpt_response,score,watermark,num_objects,cv_mask_path,sam_mask_path
0,['yellow'],purple,sunflower,https://thumbs.dreamstime.com/b/black-outline-...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
1,"['brown', 'black', 'orange']",green,tabby,https://img.recraft.ai/5P-WlUIGI2MwJC2v_JyrGkp...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
2,"['pink', 'white']",green,spoonbill,https://img.freepik.com/premium-vector/drawing...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
3,['white'],green,Samoyed,https://img.freepik.com/premium-photo/black-wh...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
4,"['grey', 'black']",red,Norwegian elkhound,https://t3.ftcdn.net/jpg/16/85/30/00/360_F_168...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",47.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
...,...,...,...,...,...,...,...,...,...,...,...
215,"['gold', 'silver']",green,crown,https://i.pinimg.com/736x/32/52/35/325235d85fb...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",42.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
216,"['orange', 'white']",purple,pill bottle,https://media.istockphoto.com/id/470313936/vec...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",35.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
217,['black'],pink,dial telephone,https://png.pngtree.com/png-clipart/20230811/o...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",35.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...
218,['black'],pink,digital watch,https://img.freepik.com/premium-vector/digital...,/mnt/lustre/work/eickhoff/esx061/color-concept...,"{'white_background': 'yes', 'style': 'illustra...",34.0,False,1.0,/mnt/lustre/work/eickhoff/esx061/color-concept...,/mnt/lustre/work/eickhoff/esx061/color-concept...


### 2.2. Get model-specific color priors

In [19]:
# Initialize GPTColorPriors
model_name = "gpt-4o"
priors_GPT = GPTColorPriors(model_name=model_name, data_folder=DATA)

# Generate GPT color priors
#priors_df_GPT = priors_GPT.get_model_color_priors(df_GPT, save=True)

# Load previously generated GPT priors
priors_df_GPT = priors_GPT.load_model_priors()

display(priors_df_GPT)

,object,correct_answer,dummy_priors,image_priors
0,sunflower,[yellow],"[yellow, brown, green]","[yellow, brown, green]"
1,tabby,"[brown, black, orange]","[brown, grey, orange]","[orange, grey, brown]"
2,spoonbill,"[pink, white]","[white, pink, black]","[white, black, grey]"
3,Samoyed,[white],"[white, cream, biscuit]","[white, cream, light grey]"
4,Norwegian elkhound,"[grey, black]","[grey, black, white]","[white, grey, black]"
...,...,...,...,...
215,crown,"[gold, silver]","[gold, silver, bronze]","[gold, silver, bronze]"
216,pill bottle,"[orange, white]","[orange, white, brown]","[brown, white, orange]"
217,dial telephone,[black],"[black, beige, red]","[black, red, beige]"
218,digital watch,[black],"[black, silver, blue]","[silver, black, blue]"


In [21]:
# Create a color prior table with one primary color per object using the IMAGE prompts
DUMMY_GPT = False
suffix = "dummy_priors" if DUMMY_GPT else "image_priors"
priors_df_GPT["prior"], priors_df_GPT["prior_changed"] = priors_GPT.pick_primary_color(priors_df_GPT, column=suffix)

[INFO] spoonbill: replaced 'white' with 'grey'
[NULL] Samoyed: all priors invalid ['white', 'cream', 'light grey'] to NaN
[INFO] Norwegian elkhound: replaced 'white' with 'grey'
[INFO] hare: replaced 'white' with 'grey'
[INFO] car wheel: replaced 'silver' with 'grey'
[INFO] dandelion: replaced 'white' with 'green'
[INFO] bat_(animal): replaced 'black' with 'brown'
[INFO] ground beetle: replaced 'black' with 'brown'
[NULL] miniature pinscher: all priors invalid ['tan', 'black', 'white'] to NaN
[INFO] Chesapeake Bay retriever: replaced 'black' with 'brown'
[INFO] gorilla: replaced 'black' with 'grey'
[INFO] giraffe: replaced 'tan' with 'brown'
[INFO] golf ball: replaced 'white' with 'yellow'
[NULL] French horn: all priors invalid ['gold', 'silver', 'brass'] to NaN
[NULL] washer: all priors invalid ['white', 'silver', 'black'] to NaN
[NULL] thimble: all priors invalid ['silver', 'gold', 'bronze'] to NaN
[NULL] bathtub: all priors invalid ['white', 'cream', 'light blue'] to NaN
[INFO] scre

In [22]:
print(f"Number of objects with valid model color priors: {len(priors_df_GPT['prior'].dropna())}")
print(f"Number of objects where the original prior was changed to be valid: {priors_df_GPT['prior_changed'].sum()}")
df_GPT["correct_answer"] = priors_df_GPT["prior"]

df_GPT.dropna(subset=["correct_answer"], axis=0, inplace=True)

#priors.analyze_differences(priors_df)
display(priors_df_GPT.dropna())
df_GPT["correct_answer"].value_counts()

Number of objects with valid model color priors: 199
Number of objects where the original prior was changed to be valid: 83


,object,correct_answer,dummy_priors,image_priors,prior,prior_changed
0,sunflower,[yellow],"[yellow, brown, green]","[yellow, brown, green]",yellow,False
1,tabby,"[brown, black, orange]","[brown, grey, orange]","[orange, grey, brown]",orange,False
2,spoonbill,"[pink, white]","[white, pink, black]","[white, black, grey]",grey,True
4,Norwegian elkhound,"[grey, black]","[grey, black, white]","[white, grey, black]",grey,True
5,lobster,[red],"[red, blue, yellow]","[red, brown, blue]",red,False
...,...,...,...,...,...,...
214,apple,"[red, green, yellow]","[red, green, yellow]","[red, green, yellow]",red,False
216,pill bottle,"[orange, white]","[orange, white, brown]","[brown, white, orange]",brown,False
217,dial telephone,[black],"[black, beige, red]","[black, red, beige]",red,True
218,digital watch,[black],"[black, silver, blue]","[silver, black, blue]",blue,True


correct_answer
brown     71
green     28
grey      28
red       27
blue      14
orange    13
yellow    11
pink       4
purple     3
Name: count, dtype: int64

In [23]:
# save filtered dataframe with GPT priors
name = f"color_priors_gpt-4o_{suffix}"
priors_df_GPT.to_csv(DATA / f"{name}.csv", index=False)

In [24]:
# Compare GPT vs Torch priors
torch_df = priors_df[["object", "prior"]].dropna().rename(columns={"prior": "prior_torch"})
gpt_df = priors_df_GPT[["object", "prior"]].dropna().rename(columns={"prior": "prior_gpt"})
valid_torch = torch_df.shape[0]
valid_gpt   = gpt_df.shape[0]
print(f"Number of objects with valid Torch priors: {valid_torch}")
print(f"Number of objects with valid GPT priors: {valid_gpt}")

merged = torch_df.merge(gpt_df, on="object", how="outer")



both_valid = merged.dropna(subset=["prior_torch", "prior_gpt"])
agreement_rate = (both_valid["prior_torch"] == both_valid["prior_gpt"]).mean()

print(f"Agreement rate (only objects with two priors): {agreement_rate:.3f}")
print(f"Compared objects: {len(both_valid)}")

diff = both_valid[both_valid["prior_torch"] != both_valid["prior_gpt"]]
print("Number of disagreements:", len(diff))
display(diff)

Number of objects with valid Torch priors: 196
Number of objects with valid GPT priors: 199
Agreement rate (only objects with two priors): 0.644
Compared objects: 188
Number of disagreements: 67


,object,prior_torch,prior_gpt
0,Band Aid,red,pink
5,Komodo dragon,green,brown
9,Old English sheepdog,brown,grey
19,anemone fish,grey,orange
20,ant,brown,red
...,...,...,...
200,walnut,grey,brown
201,walrus,grey,brown
204,wheelbarrow,red,green
205,wombat,grey,brown


### 2.3. Color black-and-white base images with color prior

In [25]:
# Upadate df_GPT with resized images and masks
df_GPT = resize_all_images_and_masks(
    df_GPT,
    img_out_folder=RESIZED_IMGS,
    mask_out_folder=RESIZED_MASKS,
    target_size=512,              # or 256, 384, 1024
    mask_column="cv_mask_path",   
    img_column="image_path"
)

Resizing all: 100%|██████████| 199/199 [00:14<00:00, 13.69it/s]

Resized 199 images.


In [26]:
# Generate variants for all images in GPT prior df
OUT_COLOR = OUTLINES / model_name / suffix
print(f"Saving to {OUT_COLOR}")
OUT_COLOR.mkdir(parents=True, exist_ok=True)

Saving to /mnt/lustre/work/eickhoff/esx061/color-concept-entanglement/data/color_images/gpt-4o/image_priors


In [29]:
for _, row in tqdm(df_GPT.iterrows(), total=len(df_GPT), desc="Generating color variants"):
        paths = generate_variants(
            row=row,
            target_color=row['correct_answer'],
            out_dir=OUT_COLOR,
            rng=rng,
            use_patches=True,
            patch_size=16,
            mode="sequential",
            pct_schedule=pct_schedule
        )

Generating color variants: 100%|██████████| 13/13 [01:41<00:00,  7.79s/it]


In [ ]:
for idx, row in df_GPT.head(5).iterrows():
    for color_mode in coloring_modes:
        show_variants_grid(row['image_path'], row['incorrect_answer'], OUT_COLOR, color_mode=color_mode, pct_range=ten_increase)